In [1]:
import config
import clr_callback
import learningratefinder

In [2]:
ls

clr_callback.py  drive/                 __pycache__/
config.py        learningratefinder.py  sample_data/


In [3]:
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


In [4]:
print(config.DATASET_PATH_COLAB)

/content/drive/My Drive/Group_Classification/input


In [7]:
#setting agg as bg as this will save the imgs in the bg
import matplotlib
matplotlib.use("Agg")

#importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from learningratefinder import LearningRateFinder
from clr_callback import CyclicLR
import config
from imutils import paths
import matplotlib.pyplot as plt 
import numpy as np
import argparse
import pickle
import cv2
import sys
import os


# ap = argparse.ArgumentParser()
# ap.add_argument("-f", "--lr-find", type=int , default= 0,
#                 help="whether or not to find optimal learning rate")

# args = vars(ap.parse_args())
args={}
args["lr_find"] = 0


print("[INFO] LOADING IMAGES...")
imagePaths = list(paths.list_images(config.DATASET_PATH_COLAB))
# print(imagePaths)
data = []
labels = []

i = 0
#looping over imagePaths
for imagePath in imagePaths:
    #extract the class label
    # print(imagePath)
    label = imagePath.split(os.path.sep)[-2]
    # print(label)
    
    #loading the image, converting it to RGB channel, and resize
    #to be fixed at 224*224 
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224,224))

    #update the data ans labels lists 
    data.append(image)
    labels.append(label)
    # print(data)
    if i %100 ==0:
        print(i)
    i+= 1

#converting the data and labels tp numpy array
print("[INFO] PROCESSING DATA...")
data = np.array(data, dtype="float32")
labels = np.array(labels)

#perform one-hot encoding on the labels 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

print(labels[0])



[INFO] LOADING IMAGES...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
[INFO] PROCESSING DATA...
[1 0 0]


In [8]:
data.shape

(3308, 224, 224, 3)

In [9]:

#partition the data into training and testing 
(trainX, testX, trainY, testY) = train_test_split(data,
                                                 labels,
                                                 test_size=config.TEST_SPLIT,
                                                 random_state = 42)
                                            

#preparing the validation split from the training split
(trainX, valX, trainY, valY) = train_test_split(trainX,
                                                trainY,
                                                test_size = config.VAL_SPLIT,
                                                random_state = 84)

#initialize the training data augmentation object 

aug = ImageDataGenerator(
            rotation_range= 30,
            zoom_range = 0.15,
            width_shift_range= 0.2,
            height_shift_range= 0.2,
            shear_range= 0.15,
            horizontal_flip= True,
            fill_mode="nearest")
    



In [10]:
trainX.shape

(2232, 224, 224, 3)

In [11]:
args["lr_find"] = 0

In [12]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet152

In [21]:
with tf.device("/gpu:0"):
    #loading the model 
    basemodel = ResNet50(weights= "imagenet",
                    include_top= False,
                    input_tensor = Input(shape=(224,224,3))
                    )

    #headmodel which will be placed on basemodel            
    headmodel = basemodel.output
    headmodel = Flatten(name="Flatten")(headmodel)
    headmodel = Dense(512, activation="relu")(headmodel)
    headmodel = Dropout(0.5)(headmodel)
    headmodel = Dense(len(config.CLASSES), activation="softmax")(headmodel)

    #placing the headmodel on top of the base model
    model = Model(inputs=basemodel.input, outputs=headmodel)

    #freezing the layers of the basemodel
    for layer in basemodel.layers:
        layer.trainable = False

    print("[INFO] COMPILING MODEL ...")

    opt = SGD(lr = config.MIN_LR, momentum= 0.9)
    model.compile(loss = "categorical_crossentropy",
                optimizer = opt,
                metrics = ["accuracy"])

    #cheacking whether we have to perform finding optimal rate
    if args["lr_find"] > 0:
        print("[INFO] finding learning rate...")
        lrf = LearningRateFinder(model)
        lrf.find(
            aug.flow(trainX, trainY, batch_size= config.BATHC_SIZE),
            1e-10, 1e+1,
            stepsPerEpoch= np.ceil((trainX.shape[0]/float(config.BATHC_SIZE))),
            epochs= 20,
            batchSize= config.BATHC_SIZE
        )

        #plotting the loss for various lrs
        lrf.plot_loss()
        plt.savefig(config.LRFIND_PLOT_PATH)

        #exiting the script so as to change the lrs in the config file
        print("[INFO] learning rate finder complete")
        print("[INFO] examine plots and adjust learning rates before training")

        #exit the script
        sys.exit(0)

    stepsize = config.STEP_SIZE *(trainX.shape[0] // config.BATHC_SIZE)
    clr =  CyclicLR(
        mode= config.CLR_METHOD,
        base_lr=config.MIN_LR,
        max_lr=config.MAX_LR,
        step_size= stepsize
    )


    print(["[INFO] training netwrok..."])
    H = model.fit(
            aug.flow(trainX, trainY, batch_size= config.BATHC_SIZE),
            validation_data = (valX, valY),
            steps_per_epoch = trainX.shape[0] // config.BATHC_SIZE,
            epochs = 25,
            callbacks = [clr]
            # verbose = 1
    )


    print("[INFO] EVALUATING THE NETWORK...")

    predictions = model.predict(testX, batch_size = config.BATHC_SIZE)

    print(classification_report(testY.argmax(axis =1),
        predictions.argmax(axis =1),
        target_names = config.CLASSES))

    print("[INFO] SERIALIZING NETWROK TO {} .... ".format(config.MODEL_PATH))
    model.save(config.MODEL_PATH_COLAB)    


    #plot that plots and saves the training history
    N = np.arange(0, 25)
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(N, H.history["loss"], label= "train_loss")
    plt.plot(N, H.history["val_loss"], label = "val_loss")
    plt.plot(N, H.history["accuracy"], label = "train_acc")
    plt.plot(N, H.history["val_accuracy"], label = "val_acc")
    plt.title("Training Loss and Accuracy")
    plt.ylabel("Loss / Accuracy")
    plt.xlabel("Epoch #")
    plt.legend(loc = "lower left")
    plt.savefig(config.TRAINING_PLOT_PATH)


    # plot the learning rate history
    N = np.arange(0, len(clr.history["lr"]))
    plt.figure()
    plt.plot(N, clr.history["lr"])
    plt.title("Cyclical Learning Rate (CLR)")
    plt.xlabel("Training Iterations")
    plt.ylabel("Learning Rate")
    plt.savefig(config.CLR_PLOT_PATH)

94773248/94765736 [==============================] - 3s 0us/step
[INFO] COMPILING MODEL ...
['[INFO] training netwrok...']
Epoch 1/25
69/69 [==============================] - 33s 480ms/step - loss: 2.4335 - accuracy: 0.3991 - val_loss: 1.0084 - val_accuracy: 0.5984
Epoch 2/25
69/69 [==============================] - 32s 463ms/step - loss: 1.3974 - accuracy: 0.5605 - val_loss: 0.7566 - val_accuracy: 0.7028
Epoch 3/25
69/69 [==============================] - 32s 463ms/step - loss: 0.9355 - accuracy: 0.6568 - val_loss: 0.6493 - val_accuracy: 0.7510
Epoch 4/25
69/69 [==============================] - 32s 463ms/step - loss: 0.7678 - accuracy: 0.6845 - val_loss: 0.5713 - val_accuracy: 0.7309
Epoch 5/25
69/69 [==============================] - 32s 461ms/step - loss: 0.6959 - accuracy: 0.7186 - val_loss: 0.4968 - val_accuracy: 0.7831
Epoch 6/25
69/69 [==============================] - 32s 463ms/step - loss: 0.5939 - accuracy: 0.7345 - val_loss: 0.4424 - val_accuracy: 0.8032
Epoch 7/25
69/69 [=

In [22]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [23]:
model.save("/content/drive/My Drive/Group_Classification/output/ResNet50model2.h5")

In [24]:
model.load_weights("/content/drive/My Drive/Group_Classification/output/ResNet50model2.h5")

In [25]:
import pandas as pd
from tqdm import tqdm

In [26]:
test  = pd.read_csv("/content/drive/My Drive/Group_Classification/output/Test.csv")

# print(test.shape)
test_img = []
# image = cv2.imread("/home/lohith/Documents/projects/Group_Classification/Test Data/Img3968.jpg")

for i in tqdm(range(test.shape[0])):
    imagePath = "/content/drive/My Drive/Group_Classification/Test Data/" + test["Filename"][i] 
    # print(imagePath)


    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224,224))

    test_img.append(image)

test = np.array(test_img)
# print(test_img[0])   



100%|██████████| 314/314 [00:02<00:00, 129.86it/s]


In [27]:
test.shape

(314, 224, 224, 3)

In [28]:
prediction = model.predict(test) 
final_pred =  lb.inverse_transform(prediction)
test  = pd.read_csv("/content/drive/My Drive/Group_Classification/output/Test.csv")
test ["Category"] = final_pred
test.Category = test.Category.map({"group of adults": "Adults", "group of babaies": "Toddler", "group of teenagers":"Teenagers"})
test.to_csv("/content/drive/My Drive/Group_Classification/output/Submission_resnet50-2.csv", index= False)